In [4]:
    import requests
    import pandas as pd
    from datetime import date

    # URL template (corrected)
    url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15"

    # Instrument tokens
    tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

    # Interval mapping
    KI = {
        "3m": "3minute",
        "5m": "5minute",
        "15m": "15minute",
        "60m": "60minute",
        "day": "day",
    }

    # Function to get candle data
    def getCandles(symbol, fromDate, toDate, timeframe):
        headers = {
            "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
            # Replace this with your actual token securely
        }
        curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
        print("Requesting:", curUrl)

        try:
            session = requests.session()
            response = session.get(curUrl, headers=headers)
            r = response.json()
        except Exception as error:
            print("API request failed:", error)
            return pd.DataFrame()

        try:
            history = r["data"]["candles"]
            history_df = pd.DataFrame(history)
            history_df = history_df.rename(
                columns={
                    0: "Datetime",
                    1: "open",
                    2: "high",
                    3: "low",
                    4: "close",
                    5: "volume",
                }
            )
            history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
            history_df.set_index("Datetime", inplace=True)
            return history_df
        except Exception as error:
            print("Data processing failed:", error)
            return pd.DataFrame()


    # === Hammer Candlestick Pattern Detector ===
    def hammer(ohlc_df):
        df = ohlc_df.copy()
        df["hammer"] = (
            ((df["high"] - df["low"]) > 3 * (df["open"] - df["close"]).abs())
            & ((df["close"] - df["low"]) / (0.001 + df["high"] - df["low"]) > 0.6)
            & ((df["open"] - df["low"]) / (0.001 + df["high"] - df["low"]) > 0.6)
            & (abs(df["close"] - df["open"]) > 0.1 * (df["high"] - df["low"]))
        )
        return df


    print("Start")
    symbol_data = getCandles("NIFTY", "2025-03-20", "2025-03-25", KI["15m"])


    if not symbol_data.empty:
        hammer_df = hammer(symbol_data)
        print(hammer_df[hammer_df["hammer"] == True])
    else:
        print("No OHLC data available.")


Start
Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15
                               open      high       low     close  volume  6  \
Datetime                                                                       
2025-06-11 09:15:00+05:30  25134.15  25136.20  25081.30  25128.20       0  0   
2025-06-11 09:40:00+05:30  25124.40  25126.20  25115.85  25126.20       0  0   
2025-06-11 10:05:00+05:30  25153.05  25158.35  25144.35  25157.45       0  0   
2025-06-11 10:25:00+05:30  25140.50  25141.50  25132.40  25138.85       0  0   
2025-06-11 12:00:00+05:30  25210.15  25213.00  25205.00  25211.90       0  0   
2025-06-11 12:15:00+05:30  25206.20  25209.15  25201.50  25207.15       0  0   
2025-06-11 15:05:00+05:30  25145.75  25149.35  25140.25  25147.95       0  0   
2025-06-12 09:45:00+05:30  25103.45  25114.45  25083.45  25108.95       0  0   
2025-06-12 10:20:00+05:30  25091.20  25098.75  25073.05  25098.05